In [1]:
%config Completer.use_jedi = False
import configparser
config = configparser.ConfigParser()
config.read('XXX-config.txt')

source_folder = config.get('Configuration', 'source_folder0')
time_step = int(config.get('Configuration', 'time_step'))
window_size = int(config.get('Configuration', 'window_size'))

ini_date = config.get('Configuration', 'ini_date0')
end_date = config.get('Configuration', 'end_date0')

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))

anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

rooms=config.get('Configuration', 'rooms')
rooms = [(item.strip()) for item in rooms.split(',')]

In [2]:
import pandas as pd
import collections
import math
import os

import cv2
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import math
import time
from PIL import Image 
import shutil

from pandas import read_csv  



In [3]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

day=list(range(day_time(t0),day_time(tN)+1))[0]
print(day)

2024-03-16 13:45:30 Init date is 1710593130 day: 19798 2024-03-16 13:45:30
2024-03-16 21:30:00 End date is 1710621000 day: 19798 2024-03-16 21:30:00
19798


In [4]:
df_total_map = {}

for room in rooms:
    df_total_map[room]={}
    print(os.listdir(os.path.join(source_folder,room)))
    for person in os.listdir(os.path.join(source_folder,room)):
        df_total_map[room][person]={}
        for scene in os.listdir(os.path.join(source_folder,room,person)):
            print("scene",scene)
            df_total_map[room][person][scene]=(None,None)
            for user in users:   
                print(os.path.join(source_folder,room,person,scene)+"/location_gt_"+user+"/"+str(day)+".location.tsv")
                try:
                    series=read_csv(os.path.join(source_folder,room,person,scene)+"/location_gt_"+user+"/"+str(day)+".location.tsv", sep="\t",parse_dates=True, header=None, names=["time","date","time2","x","y"])
                    tA,tB=series["time"].iloc[0],series["time"].iloc[-1]
                    print(series.size,(series["time"].iloc[0],series["time"].iloc[-1]))
                    if(df_total_map[room][person][scene]==(None,None)):
                        print("starting ",room,person,scene, "from ",df_total_map[room][person][scene],(tA,tB))
                        df_total_map[room][person][scene]=(tA,tB)
                    else:
                        print("updating ",room,person,scene, "from ",df_total_map[room][person][scene])
                        tA=min(df_total_map[room][person][scene][0],tA)
                        tB=max(df_total_map[room][person][scene][1],tB)
                        df_total_map[room][person][scene]=(tA,tB)
                        print("updating ",room,person,scene, "to  ",df_total_map[room][person][scene])
                except Exception as e:
                    print("Not data",e)
                    continue



['1-person-labelled', '2-person-labelled', '3-person-labelled']
scene Scene1
./data/livingroom\1-person-labelled\Scene1/location_gt_f02e/19798.location.tsv
Not data [Errno 2] No such file or directory: './data/livingroom\\1-person-labelled\\Scene1/location_gt_f02e/19798.location.tsv'
./data/livingroom\1-person-labelled\Scene1/location_gt_867f/19798.location.tsv
Not data [Errno 2] No such file or directory: './data/livingroom\\1-person-labelled\\Scene1/location_gt_867f/19798.location.tsv'
./data/livingroom\1-person-labelled\Scene1/location_gt_ed9c/19798.location.tsv
1020 (1710593157, 1710593451)
starting  livingroom 1-person-labelled Scene1 from  (None, None) (1710593157, 1710593451)
scene Scene2
./data/livingroom\1-person-labelled\Scene2/location_gt_f02e/19798.location.tsv
Not data [Errno 2] No such file or directory: './data/livingroom\\1-person-labelled\\Scene2/location_gt_f02e/19798.location.tsv'
./data/livingroom\1-person-labelled\Scene2/location_gt_867f/19798.location.tsv
Not data

Not data [Errno 2] No such file or directory: './data/kitchen\\1-person-labelled\\Scene3/location_gt_867f/19798.location.tsv'
./data/kitchen\1-person-labelled\Scene3/location_gt_ed9c/19798.location.tsv
1345 (1710616574, 1710616872)
starting  kitchen 1-person-labelled Scene3 from  (None, None) (1710616574, 1710616872)
scene Scene4
./data/kitchen\1-person-labelled\Scene4/location_gt_f02e/19798.location.tsv
Not data [Errno 2] No such file or directory: './data/kitchen\\1-person-labelled\\Scene4/location_gt_f02e/19798.location.tsv'
./data/kitchen\1-person-labelled\Scene4/location_gt_867f/19798.location.tsv
Not data [Errno 2] No such file or directory: './data/kitchen\\1-person-labelled\\Scene4/location_gt_867f/19798.location.tsv'
./data/kitchen\1-person-labelled\Scene4/location_gt_ed9c/19798.location.tsv
1500 (1710617078, 1710617377)
starting  kitchen 1-person-labelled Scene4 from  (None, None) (1710617078, 1710617377)
scene Scene5
./data/kitchen\1-person-labelled\Scene5/location_gt_f02e/1

In [5]:
print(df_total_map)

{'livingroom': {'1-person-labelled': {'Scene1': (1710593157, 1710593451), 'Scene2': (1710593672, 1710593967), 'Scene3': (1710594542, 1710594853), 'Scene4': (1710594999, 1710595299), 'Scene5': (1710595474, 1710595770), 'Scene6': (1710595940, 1710596236)}, '2-person-labelled': {'Scene1': (1710613851, 1710614183), 'Scene2': (1710614422, 1710614779), 'Scene3': (1710615280, 1710615624), 'Scene4': (1710615774, 1710616088), 'Scene5': (1710616605, 1710616905), 'Scene6': (1710617076, 1710617196)}, '3-person-labelled': {'Scene1': (1710622214, 1710622865), 'Scene2': (1710622440, 1710622679), 'Scene3': (1710622679, 1710622878)}}, 'kitchen': {'1-person-labelled': {'Scene1': (1710615095, 1710615399), 'Scene2': (1710615622, 1710615916), 'Scene3': (1710616574, 1710616872), 'Scene4': (1710617078, 1710617377), 'Scene5': (1710617644, 1710617941), 'Scene6': (1710618066, 1710618371)}, '2-person-labelled': {'Scene1': (1710621078, 1710621359), 'Scene2': (1710621360, 1710621664)}, '3-person-labelled': {'Scene

In [6]:
def isSceneTime(tt):
    for room in df_total_map:
        for person in df_total_map[room]:
            for scene in sorted(df_total_map[room][person].keys(), reverse=True): 
                (tA,tB)=df_total_map[room][person][scene]
                #print("\t",(tA,tB))
                if (tA,tB) == (None,None):
                    continue
                if tt>=tA and tt<=tB:
                    return scene
    return None

In [7]:
tsN={}
tRef=0
with open(os.path.join(source_folder+"-aligned","scene-time.txt"), 'w') as file:
    for tt in range(t0,tN,time_step):
        scene=isSceneTime(tt)
        #print(tt,scene)
        if(scene):
            file.write(str(tt)+"\t"+str(tRef)+"\t"+scene+"\n")
            tsN[tt]=tRef
            tRef=tRef+1
    

In [8]:
print(tsN)

{1710593157: 0, 1710593158: 1, 1710593159: 2, 1710593160: 3, 1710593161: 4, 1710593162: 5, 1710593163: 6, 1710593164: 7, 1710593165: 8, 1710593166: 9, 1710593167: 10, 1710593168: 11, 1710593169: 12, 1710593170: 13, 1710593171: 14, 1710593172: 15, 1710593173: 16, 1710593174: 17, 1710593175: 18, 1710593176: 19, 1710593177: 20, 1710593178: 21, 1710593179: 22, 1710593180: 23, 1710593181: 24, 1710593182: 25, 1710593183: 26, 1710593184: 27, 1710593185: 28, 1710593186: 29, 1710593187: 30, 1710593188: 31, 1710593189: 32, 1710593190: 33, 1710593191: 34, 1710593192: 35, 1710593193: 36, 1710593194: 37, 1710593195: 38, 1710593196: 39, 1710593197: 40, 1710593198: 41, 1710593199: 42, 1710593200: 43, 1710593201: 44, 1710593202: 45, 1710593203: 46, 1710593204: 47, 1710593205: 48, 1710593206: 49, 1710593207: 50, 1710593208: 51, 1710593209: 52, 1710593210: 53, 1710593211: 54, 1710593212: 55, 1710593213: 56, 1710593214: 57, 1710593215: 58, 1710593216: 59, 1710593217: 60, 1710593218: 61, 1710593219: 62, 1

In [9]:
import os

for room in rooms:
    for person in os.listdir(os.path.join(source_folder,room)):
        for scene in os.listdir(os.path.join(source_folder,room,person)):
            for sensors in os.listdir(os.path.join(source_folder,room,person,scene)):   
                for files in os.listdir(os.path.join(source_folder,room,person,scene,sensors)): 
                    print(os.path.join(source_folder,room,person,scene,sensors,files))
                    print(os.path.join(source_folder+"-aligned",room,person,scene,sensors))
                    os.makedirs(os.path.join(source_folder+"-aligned",room,person,scene,sensors), exist_ok=True)
                    with open(os.path.join(source_folder,room,person,scene,sensors,files), 'r') as file,  open(os.path.join(source_folder+"-aligned",room,person,scene,sensors,files), 'w') as output_file:
                        for line in file:
                            try:
                                datas=line.split("\t")
                                #print(datas)
                                tt=int(datas[0])
                                if not tt in tsN:
                                    continue
                                datas[0]=str(tsN[tt]+315576000)
                                datas[1]=time2str(tsN[tt]+315576000)
                                #print("*",datas,"*")
                                output_file.write(datas[0]+"\t"+datas[1])
                                for data in datas[2:]:
                                    output_file.write("\t"+data)
                                #output_file.write("\n")
                            except:
                                print("@ bad line",line)
                    
                        
                        
                

./data/livingroom\1-person-labelled\Scene1\binary\19798.s1.tsv
./data/-aligned\livingroom\1-person-labelled\Scene1\binary
./data/livingroom\1-person-labelled\Scene1\binary\19798.s10.tsv
./data/-aligned\livingroom\1-person-labelled\Scene1\binary
./data/livingroom\1-person-labelled\Scene1\binary\19798.s11.tsv
./data/-aligned\livingroom\1-person-labelled\Scene1\binary
./data/livingroom\1-person-labelled\Scene1\binary\19798.s12.tsv
./data/-aligned\livingroom\1-person-labelled\Scene1\binary
./data/livingroom\1-person-labelled\Scene1\binary\19798.s13.tsv
./data/-aligned\livingroom\1-person-labelled\Scene1\binary
./data/livingroom\1-person-labelled\Scene1\binary\19798.s14.tsv
./data/-aligned\livingroom\1-person-labelled\Scene1\binary
./data/livingroom\1-person-labelled\Scene1\binary\19798.s15.tsv
./data/-aligned\livingroom\1-person-labelled\Scene1\binary
./data/livingroom\1-person-labelled\Scene1\binary\19798.s16.tsv
./data/-aligned\livingroom\1-person-labelled\Scene1\binary
./data/livingroom

./data/livingroom\1-person-labelled\Scene4\binary\19798.s7.tsv
./data/-aligned\livingroom\1-person-labelled\Scene4\binary
./data/livingroom\1-person-labelled\Scene4\binary\19798.s8.tsv
./data/-aligned\livingroom\1-person-labelled\Scene4\binary
./data/livingroom\1-person-labelled\Scene4\binary\19798.s9.tsv
./data/-aligned\livingroom\1-person-labelled\Scene4\binary
./data/livingroom\1-person-labelled\Scene4\location_gt_ed9c\19798.location.tsv
./data/-aligned\livingroom\1-person-labelled\Scene4\location_gt_ed9c
./data/livingroom\1-person-labelled\Scene4\ubitrack-uwb\19798.ed9c.rssi.tsv
./data/-aligned\livingroom\1-person-labelled\Scene4\ubitrack-uwb
./data/livingroom\1-person-labelled\Scene4\ubitrack-uwb\19798.ed9c.sourcedata.tsv
./data/-aligned\livingroom\1-person-labelled\Scene4\ubitrack-uwb
./data/livingroom\1-person-labelled\Scene5\binary\19798.s1.tsv
./data/-aligned\livingroom\1-person-labelled\Scene5\binary
./data/livingroom\1-person-labelled\Scene5\binary\19798.s10.tsv
./data/-alig

./data/livingroom\2-person-labelled\Scene1\binary\19798.s15.tsv
./data/-aligned\livingroom\2-person-labelled\Scene1\binary
./data/livingroom\2-person-labelled\Scene1\binary\19798.s16.tsv
./data/-aligned\livingroom\2-person-labelled\Scene1\binary
./data/livingroom\2-person-labelled\Scene1\binary\19798.s17.tsv
./data/-aligned\livingroom\2-person-labelled\Scene1\binary
./data/livingroom\2-person-labelled\Scene1\binary\19798.s18.tsv
./data/-aligned\livingroom\2-person-labelled\Scene1\binary
./data/livingroom\2-person-labelled\Scene1\binary\19798.s19.tsv
./data/-aligned\livingroom\2-person-labelled\Scene1\binary
./data/livingroom\2-person-labelled\Scene1\binary\19798.s2.tsv
./data/-aligned\livingroom\2-person-labelled\Scene1\binary
./data/livingroom\2-person-labelled\Scene1\binary\19798.s20.tsv
./data/-aligned\livingroom\2-person-labelled\Scene1\binary
./data/livingroom\2-person-labelled\Scene1\binary\19798.s21.tsv
./data/-aligned\livingroom\2-person-labelled\Scene1\binary
./data/livingroom

./data/livingroom\2-person-labelled\Scene2\binary\19798.s27.tsv
./data/-aligned\livingroom\2-person-labelled\Scene2\binary
./data/livingroom\2-person-labelled\Scene2\binary\19798.s28.tsv
./data/-aligned\livingroom\2-person-labelled\Scene2\binary
./data/livingroom\2-person-labelled\Scene2\binary\19798.s3.tsv
./data/-aligned\livingroom\2-person-labelled\Scene2\binary
./data/livingroom\2-person-labelled\Scene2\binary\19798.s4.tsv
./data/-aligned\livingroom\2-person-labelled\Scene2\binary
./data/livingroom\2-person-labelled\Scene2\binary\19798.s5.tsv
./data/-aligned\livingroom\2-person-labelled\Scene2\binary
./data/livingroom\2-person-labelled\Scene2\binary\19798.s6.tsv
./data/-aligned\livingroom\2-person-labelled\Scene2\binary
./data/livingroom\2-person-labelled\Scene2\binary\19798.s7.tsv
./data/-aligned\livingroom\2-person-labelled\Scene2\binary
./data/livingroom\2-person-labelled\Scene2\binary\19798.s8.tsv
./data/-aligned\livingroom\2-person-labelled\Scene2\binary
./data/livingroom\2-pe

./data/livingroom\2-person-labelled\Scene4\binary\19798.c26.tsv
./data/-aligned\livingroom\2-person-labelled\Scene4\binary
./data/livingroom\2-person-labelled\Scene4\binary\19798.c27.tsv
./data/-aligned\livingroom\2-person-labelled\Scene4\binary
./data/livingroom\2-person-labelled\Scene4\binary\19798.c28.tsv
./data/-aligned\livingroom\2-person-labelled\Scene4\binary
./data/livingroom\2-person-labelled\Scene4\binary\19798.c29.tsv
./data/-aligned\livingroom\2-person-labelled\Scene4\binary
./data/livingroom\2-person-labelled\Scene4\binary\19798.c3.tsv
./data/-aligned\livingroom\2-person-labelled\Scene4\binary
./data/livingroom\2-person-labelled\Scene4\binary\19798.c30.tsv
./data/-aligned\livingroom\2-person-labelled\Scene4\binary
./data/livingroom\2-person-labelled\Scene4\binary\19798.c6.tsv
./data/-aligned\livingroom\2-person-labelled\Scene4\binary
./data/livingroom\2-person-labelled\Scene4\binary\19798.c7.tsv
./data/-aligned\livingroom\2-person-labelled\Scene4\binary
./data/livingroom\2

...

./data/bathroom\1-person-labelled\Scene3\binary\19798.b11.tsv
./data/-aligned\bathroom\1-person-labelled\Scene3\binary
./data/bathroom\1-person-labelled\Scene3\binary\19798.b12.tsv
./data/-aligned\bathroom\1-person-labelled\Scene3\binary
./data/bathroom\1-person-labelled\Scene3\binary\19798.b13.tsv
./data/-aligned\bathroom\1-person-labelled\Scene3\binary
./data/bathroom\1-person-labelled\Scene3\binary\19798.b14.tsv
./data/-aligned\bathroom\1-person-labelled\Scene3\binary
./data/bathroom\1-person-labelled\Scene3\binary\19798.b15.tsv
./data/-aligned\bathroom\1-person-labelled\Scene3\binary
./data/bathroom\1-person-labelled\Scene3\binary\19798.b16.tsv
./data/-aligned\bathroom\1-person-labelled\Scene3\binary
./data/bathroom\1-person-labelled\Scene3\binary\19798.b17.tsv
./data/-aligned\bathroom\1-person-labelled\Scene3\binary
./data/bathroom\1-person-labelled\Scene3\binary\19798.b18.tsv
./data/-aligned\bathroom\1-person-labelled\Scene3\binary
./data/bathroom\1-person-labelled\Scene3\binary\

./data/bathroom\2-person-labelled\Scene1\binary\19798.s11.tsv
./data/-aligned\bathroom\2-person-labelled\Scene1\binary
./data/bathroom\2-person-labelled\Scene1\binary\19798.s12.tsv
./data/-aligned\bathroom\2-person-labelled\Scene1\binary
./data/bathroom\2-person-labelled\Scene1\binary\19798.s13.tsv
./data/-aligned\bathroom\2-person-labelled\Scene1\binary
./data/bathroom\2-person-labelled\Scene1\binary\19798.s14.tsv
./data/-aligned\bathroom\2-person-labelled\Scene1\binary
./data/bathroom\2-person-labelled\Scene1\binary\19798.s15.tsv
./data/-aligned\bathroom\2-person-labelled\Scene1\binary
./data/bathroom\2-person-labelled\Scene1\binary\19798.s16.tsv
./data/-aligned\bathroom\2-person-labelled\Scene1\binary
./data/bathroom\2-person-labelled\Scene1\binary\19798.s17.tsv
./data/-aligned\bathroom\2-person-labelled\Scene1\binary
./data/bathroom\2-person-labelled\Scene1\binary\19798.s18.tsv
./data/-aligned\bathroom\2-person-labelled\Scene1\binary
./data/bathroom\2-person-labelled\Scene1\binary\

./data/bathroom\2-person-labelled\Scene2\binary\19798.s28.tsv
./data/-aligned\bathroom\2-person-labelled\Scene2\binary
./data/bathroom\2-person-labelled\Scene2\binary\19798.s3.tsv
./data/-aligned\bathroom\2-person-labelled\Scene2\binary
./data/bathroom\2-person-labelled\Scene2\binary\19798.s4.tsv
./data/-aligned\bathroom\2-person-labelled\Scene2\binary
./data/bathroom\2-person-labelled\Scene2\binary\19798.s5.tsv
./data/-aligned\bathroom\2-person-labelled\Scene2\binary
./data/bathroom\2-person-labelled\Scene2\binary\19798.s6.tsv
./data/-aligned\bathroom\2-person-labelled\Scene2\binary
./data/bathroom\2-person-labelled\Scene2\binary\19798.s7.tsv
./data/-aligned\bathroom\2-person-labelled\Scene2\binary
./data/bathroom\2-person-labelled\Scene2\binary\19798.s8.tsv
./data/-aligned\bathroom\2-person-labelled\Scene2\binary
./data/bathroom\2-person-labelled\Scene2\binary\19798.s9.tsv
./data/-aligned\bathroom\2-person-labelled\Scene2\binary
./data/bathroom\2-person-labelled\Scene2\location_gt_86

./data/bathroom\2-person-labelled\Scene3\binary\19798.presence_sensor_fp2_6f1e_presence_sensor_23.tsv
./data/-aligned\bathroom\2-person-labelled\Scene3\binary
./data/bathroom\2-person-labelled\Scene3\binary\19798.presence_sensor_fp2_6f1e_presence_sensor_24.tsv
./data/-aligned\bathroom\2-person-labelled\Scene3\binary
./data/bathroom\2-person-labelled\Scene3\binary\19798.presence_sensor_fp2_6f1e_presence_sensor_25.tsv
./data/-aligned\bathroom\2-person-labelled\Scene3\binary
./data/bathroom\2-person-labelled\Scene3\binary\19798.presence_sensor_fp2_6f1e_presence_sensor_26.tsv
./data/-aligned\bathroom\2-person-labelled\Scene3\binary
./data/bathroom\2-person-labelled\Scene3\binary\19798.presence_sensor_fp2_6f1e_presence_sensor_27.tsv
./data/-aligned\bathroom\2-person-labelled\Scene3\binary
./data/bathroom\2-person-labelled\Scene3\binary\19798.presence_sensor_fp2_6f1e_presence_sensor_28.tsv
./data/-aligned\bathroom\2-person-labelled\Scene3\binary
./data/bathroom\2-person-labelled\Scene3\binar

./data/bathroom\2-person-labelled\Scene4\ubitrack-uwb\19798.867f.sourcedata.tsv
./data/-aligned\bathroom\2-person-labelled\Scene4\ubitrack-uwb
./data/bathroom\2-person-labelled\Scene4\ubitrack-uwb\19798.ed9c.rssi.tsv
./data/-aligned\bathroom\2-person-labelled\Scene4\ubitrack-uwb
./data/bathroom\2-person-labelled\Scene4\ubitrack-uwb\19798.ed9c.sourcedata.tsv
./data/-aligned\bathroom\2-person-labelled\Scene4\ubitrack-uwb
./data/bathroom\2-person-labelled\Scene4\ubitrack-uwb\19798.f02e.rssi.tsv
./data/-aligned\bathroom\2-person-labelled\Scene4\ubitrack-uwb
./data/bathroom\2-person-labelled\Scene4\ubitrack-uwb\19798.f02e.sourcedata.tsv
./data/-aligned\bathroom\2-person-labelled\Scene4\ubitrack-uwb
./data/bathroom\2-person-labelled\Scene5\binary\19798.b1.tsv
./data/-aligned\bathroom\2-person-labelled\Scene5\binary
./data/bathroom\2-person-labelled\Scene5\binary\19798.b10.tsv
./data/-aligned\bathroom\2-person-labelled\Scene5\binary
./data/bathroom\2-person-labelled\Scene5\binary\19798.b11.ts

In [10]:
time2str(315576000),time2str(tRef+315576000)

('1980-01-01 13:00:00', '1980-01-01 14:51:24')